In [22]:
# opens list of tags associated with pro-recovery (labeled pos) or pro-eating disorder (labeled neg)
f = open("postags.txt", "r")
tagslist = []
for tag in f:
    tagslist.append(tag.lower().replace('\n',''))
f.close()

['eating disorder recovery', 'ed recovery', 'anorexia recovery', 'ana recovery', 'bulimia recovery', 'ednos recovery', 'pro recovery', 'body positivity', 'recovery', 'positive', 'positivity', 'self love', 'self care']


In [23]:
# --- IMPORTS ---
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
import string
import nltk

#create new file
writefile = open("posposts.txt", "w")

#instantiate webdriver
driver = webdriver.Chrome('/Users/lionheart/anaconda3/pkgs/chromedriver-binary-2.38-0/bin/chromedriver-binary')

for tag in tagslist:
    driver.get('https://www.tumblr.com/search/' + tag)
    
    # click link to find results, if applicable
    try:
        link = driver.find_element_by_link_text('View search results')
        link.click()
    except NoSuchElementException:
        pass

    # Selenium script for infinite scroll; scrolls down a maximum of 10 times for each tag
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    counter = 10
    while(match==False or counter > 0):
        lastCount = lenOfPage
        time.sleep(5)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True
        counter -= 1

    # retrieves source code
    source_data = driver.page_source
    
    soup = BeautifulSoup(source_data, "html.parser")
    
    # retrieves all information contained in the body of the post
    posts = soup.findAll("div", {"class": "post_body"})

    # writes every post for each of pos or neg to a file
    for x in posts:
        posttext = x.text
        writefile.write(posttext)
        writefile.write("\n")

writefile.close()